<a href="https://colab.research.google.com/github/Fatulla/AILAB_TASK01/blob/main/PCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import joblib
import matplotlib.pyplot as plt

In [63]:
data = pd.read_csv("Cars_dataset.csv")

In [64]:
data.head(20)

,Brand,City,Prod_Year,Ban_Type,Color,Mileage,Crashed,Price_clean,IsShop,EngineVolume,...,Park radarı,Kondisioner,Oturacaqların isidilməsi,Dəri salon,Ksenon lampalar,Arxa görüntü kamerası,Yan pərdələr,Oturacaqların ventilyasiyası,Brand_Model,Seats_Count
0,Abarth,Bakı,2012,Hetçbek,Qara,163000,0,17300,0.0,1.4,...,0,1,0,1,1,0,0,0,Abarth 595,3
1,Abarth,Bakı,2013,Hetçbek,Qara,150000,0,18000,1.0,1.4,...,1,1,0,0,1,1,0,0,Abarth 595,3
2,Abarth,Bakı,2017,Hetçbek,Yaş Asfalt,56500,0,29750,0.0,1.4,...,1,1,0,0,1,1,1,0,Abarth 595,3
3,Acura,Bakı,2011,Offroader / SUV,Ağ,70000,0,32300,1.0,3.7,...,0,1,1,1,1,1,0,1,Acura MDX,6
4,Acura,Bakı,2012,Offroader / SUV,Ağ,69900,0,33150,0.0,3.7,...,1,1,1,1,1,1,0,1,Acura MDX,6
5,Alfa Romeo,Bakı,2017,Kupe,Ağ,10000,0,93500,0.0,1.7,...,1,1,1,1,1,0,0,0,Alfa Romeo 4C,1
6,Alfa Romeo,Bakı,2020,Sedan,Qara,32817,0,64600,0.0,2.0,...,1,1,0,1,1,1,0,0,Alfa Romeo Giulia,4
7,Alfa Romeo,Bakı,2022,Sedan,Qara,0,0,91136,1.0,2.0,...,1,1,1,1,1,1,0,1,Alfa Romeo Giulia,3
8,Alfa Romeo,Bakı,2022,Sedan,Qara,0,0,156696,1.0,2.9,...,1,1,1,1,1,1,1,0,Alfa Romeo Giulia,4
9,Alfa Romeo,Bakı,2017,Hetçbek,Boz,42000,0,20000,0.0,1.4,...,0,1,0,1,0,0,0,0,Alfa Romeo Mito,4


In [65]:
# 1. Lazımi sütunları seçirik
features = ['EngineVolume', 'EngineForce', 'Prod_Year']
target = 'Price_clean'
data = data[features + [target]].dropna()

# 2. Outlier-lərin aradan qaldırılması üçün: target dəyərlərinin paylanmasına baxıb
# Aşağı və yuxarı 1%-lik kəsimi tətbiq edirik.
lower_bound = data[target].quantile(0.01)
upper_bound = data[target].quantile(0.99)
data = data[(data[target] >= lower_bound) & (data[target] <= upper_bound)]

# 3. Target üçün log transformasiya tətbiq edirik ki, qiymətlər daha normallaşsın
data['log_price'] = np.log(data[target])

# 4. X və y bölürük
X = data[features]
y = data['log_price']

# 5. Məlumatı təlim və test hissələrinə ayırırıq
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 6. Pipeline qururuq: standartlaşdırma, PCA və Linear Regression
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    # PCA: 3 dəyərdən 2 əsas komponent çıxarırıq. İstəyə görə n_components dəyişə bilər.
    ('pca', PCA(n_components=2)),
    ('lr', LinearRegression())
])


# 7. Modeli təlim edirik
pipeline.fit(X_train, y_train)

# 8. Modelin performansını qiymətləndiririk
score = pipeline.score(X_test, y_test)
print(f"Test dəqiqliyi: {score:.2f}")

# 9. Modeli fayla qeyd edirik
joblib.dump(pipeline, 'car_price_model.pkl')
print("Model 'car_price_model.pkl' faylına qeyd olundu.")

# 10. İstifadəçi interfeysi: İstifadəçidən məlumat alaraq log qiyməti proqnozlaşdırırıq
def predict_price():
    try:
        engine_volume = float(input("EngineVolume dəyərini daxil edin: "))
        engine_force = float(input("EngineForce dəyərini daxil edin: "))
        prod_year = float(input("Prod_Year dəyərini daxil edin: "))
    except ValueError:
        print("Xəta: Düzgün rəqəm daxil edin!")
        return

    # İstifadəçi daxil etdiyi dəyərlərdən DataFrame yaradırıq
    user_input = pd.DataFrame({
        'EngineVolume': [engine_volume],
        'EngineForce': [engine_force],
        'Prod_Year': [prod_year]
    })

    # Modeli yükləyirik və log proqnoz veririk
    model = joblib.load('car_price_model.pkl')
    log_predicted_price = model.predict(user_input)

    # Log qiyməti eksponentasiya edərək orijinal qiymətə çeviririk
    predicted_price = np.exp(log_predicted_price)
    print(f"Təxmin edilən bazar dəyəri: {predicted_price[0]:.2f}")

if __name__ == '__main__':
    predict_price()


Test dəqiqliyi: 0.75
Model 'car_price_model.pkl' faylına qeyd olundu.
EngineVolume dəyərini daxil edin: 1.2
EngineForce dəyərini daxil edin: 76
Prod_Year dəyərini daxil edin: 1993
Təxmin edilən bazar dəyəri: 4389.32
